In [1]:
import pandas as pd

In [2]:
def get_useful_columns(df):
    new_df = df[["Departure", "Return", "Departure station", "Return station"]]
    return new_df

In [3]:
def process_df():
    dfs = []
    
    for y in range(2017, 2022):
        tmp_df = pd.read_csv(f"../../data/Mobi_System_data_{y}.csv", compression='zip')
        dfs.append(get_useful_columns(tmp_df))
    
    df = pd.concat(dfs)
    df["Departure"] = pd.to_datetime(df["Departure"], errors="coerce")
    df["Return"] = pd.to_datetime(df["Return"], errors="coerce")
    df.dropna(inplace=True)

    df["DepartureYear"] = df["Departure"].dt.year
    df["DepartureMonth"] = df["Departure"].dt.month
    df["ReturnYear"] = df["Return"].dt.year
    df["ReturnMonth"] = df["Return"].dt.month
    df.drop(columns=["Departure", "Return"], inplace=True)
    
    return df

In [4]:
df = process_df()

In [9]:
def get_routes(df, top_n, month, year):
    df = df[(df["DepartureMonth"] == month) & (df["ReturnMonth"] == month)]
    df = df[(df["DepartureYear"] == year) & (df["ReturnYear"] == year)]
    
    df = df.groupby(["Departure station", "Return station"]).size().reset_index(name="Count")
    df = df.sort_values(by="Count", ascending=False).set_index("Count")
    df = df.head(top_n)
    return df.to_dict()

In [10]:
get_routes(df, 5, 3, 2017)

{'Departure station': {109: '0035 Coal Harbour Community Centre',
  82: '0093 Chilco & Robson',
  62: '0187 Bute & Davie',
  58: '0050 Bute & Comox'},
 'Return station': {109: '0035 Coal Harbour Community Centre',
  82: '0093 Chilco & Robson',
  62: '0036 Bute & Robson',
  58: '0032 Comox & Denman'}}